In [ ]:
import os
import numpy as np
import jax.numpy as jnp
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt
import pickle as pkl

In [ ]:
# Set the default font to Times New Roman
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman']

In [ ]:
def calculate_coverage(directory, true_param):
    # true_param = jnp.array([0.0])
    # directory = '../res/misspec_ma1/rsnl/'
    sub_dirs = [x[0] for x in os.walk(directory)]
    sub_dirs = sub_dirs[1:]
    total_coverage = np.zeros(21)
    count_successful = 0
    for sub_dir in sub_dirs:
        try:
            with open(f'{sub_dir}/thetas.pkl', 'rb') as f:
                print('sub_dir: ', sub_dir)
                thetas = jnp.array(pkl.load(f))
                thetas = jnp.concatenate(thetas, axis=0)
                thetas = jnp.transpose(thetas)
                thetas_kernel = thetas[:, ::10]  # THIN BY 10
                # plt.hist(thetas_kernel, bins=30)
                # plt.show()
                thetas_kernel_eval = thetas[:, 1::10]  # THIN BY 10
                kde = gaussian_kde(thetas_kernel, bw_method='silverman')
                true_theta_pdf = kde.logpdf(true_param)
                print('true_theta_pdf: ', true_theta_pdf)
                theta_draws_pdf = kde.logpdf(thetas_kernel_eval)
                theta_draws_pdf = jnp.sort(theta_draws_pdf)[::-1]
                # test_xs = jnp.linspace(0.8, 1.2, 1000)
                # test_ys = kde.logpdf(test_xs)
                # plt.plot(test_xs, test_ys)
                # plt.axvline(x=true_param, color='red')
                # plt.show()
                N = len(theta_draws_pdf)
                print('N: ', N)
                x = jnp.linspace(0, 1, 21)
                indices = round(x * N) - 1
                print('indices: ', indices)
                print('theta_draws_pdf[indices]: ', theta_draws_pdf[indices])
                coverage = theta_draws_pdf[indices] < true_theta_pdf
                coverage = np.array(coverage, dtype=int)
                coverage[0] = 0
                coverage[-1] = 1
                total_coverage += coverage
                count_successful += 1
                print('coverage: ', coverage)
        except Exception as e:
            print(e)
            continue
    print('count_successful: ', count_successful)
    total_coverage = total_coverage / count_successful
    return total_coverage

In [ ]:
def plot_and_save_coverages(empirical_coverage_rsnl=None, empirical_coverage_snl=None,
                            title="",
                            folder_name=""):
    """Plot coverage."""
    plt.clf()
    plt.plot([0, 1], [0, 1], color='gray', linestyle='dashed')
    if empirical_coverage_rsnl is not None:
        plt.plot(np.linspace(0, 1, len(empirical_coverage_rsnl)), empirical_coverage_rsnl,
                label='RSNL')
    if empirical_coverage_snl is not None:
        plt.plot(np.linspace(0, 1, len(empirical_coverage_snl)), empirical_coverage_snl,
                label='SNL', linestyle='dashed')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.xticks([0, 1], fontsize=35)
    plt.yticks([0, 1], fontsize=35)
    plt.xlabel("Credibility level", fontsize=35)
    plt.ylabel("Empirical coverage", fontsize=35)
    plt.legend(fontsize=35, borderpad=0.1, labelspacing=0.1, handletextpad=0.1)
    plt.title(title)
    plt.tight_layout()
    plt.savefig(f"{folder_name}empirical_coverage.pdf", bbox_inches='tight')


In [ ]:
true_param = jnp.array([1.0])

directory = '../res/contaminated_normal/rsnl/'
empirical_coverage_rsnl = calculate_coverage(directory, true_param)

directory = '../res/contaminated_normal/snl/'
empirical_coverage_snl = calculate_coverage(directory, true_param)

In [ ]:
plot_and_save_coverages(empirical_coverage_rsnl, empirical_coverage_snl,
                        # title="Contaminated normal"
                        folder_name='contaminated_normal_'
                        )

In [ ]:
true_param = jnp.array([0.0])

directory = '../res/misspec_ma1/rsnl/'
empirical_coverage_rsnl = calculate_coverage(directory, true_param)

directory = '../res/misspec_ma1/snl/'
empirical_coverage_snl = calculate_coverage(directory, true_param)

plot_and_save_coverages(empirical_coverage_rsnl, empirical_coverage_snl,
                        folder_name='misspec_ma1_'
                        )

In [ ]:
# true_param = jnp.array([0.7, -2.9, -1.0, -0.9, 0.6])

# directory = '../res/contaminated_slcp/rsnl/'
# empirical_coverage_rsnl = calculate_coverage(directory, true_param)

# directory = '../res/contaminated_slcp/snl/'
# empirical_coverage_snl = calculate_coverage(directory, true_param)

# plot_and_save_coverages(empirical_coverage_rsnl, empirical_coverage_snl,
#                         # title="Contaminated SLCP",
#                         folder_name='contaminated_slcp_'
#                         )

In [ ]:
directory = '../res/sir/rsnl/'
true_param = jnp.array([.1, .15])
empirical_coverage_rsnl = calculate_coverage(directory, true_param)

directory = '../res/sir/snl/'
empirical_coverage_snl = calculate_coverage(directory, true_param)
plot_and_save_coverages(empirical_coverage_rsnl, empirical_coverage_snl,
                        folder_name='sir_'
                        # title="SIR"
                        )

In [ ]:
directory = "../res/contaminated_normal/rsnl_well_specified/"
true_param = jnp.array([1.0])
empirical_coverage_rsnl_well_specified = calculate_coverage(directory,
                                                            true_param)

directory = "../res/contaminated_normal/snl_well_specified/"
empirical_coverage_snl_well_specified = calculate_coverage(directory,
                                                           true_param)


In [ ]:
plot_and_save_coverages(empirical_coverage_rsnl_well_specified,
                        empirical_coverage_snl_well_specified,
                        folder_name='well_specified_normal_'
                        # title="SIR"
                        )
